In [1]:
# Import required modules
import pandas as pd
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
import time
from datetime import datetime
today= datetime.today().date().strftime("%d_%b")

In [2]:
# Initialize driver
driver = webdriver.Chrome(r"/home/faysal/Documents/softwares/chromedriver")
driver.maximize_window()
driver.get("https://www.ntu.ac.uk/about-us/staff-directory?&&start_rank=19.18181818181818")
wait = WebDriverWait(driver, 10)

# Load 10 pages only
for _ in range(10):
    try:
        element = driver.find_element_by_css_selector("a.more-results")
        webdriver.ActionChains(driver).move_to_element(element).click(element).perform()
        time.sleep(10)
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    except:
        print("no more page to load")

In [3]:
# Scrape required information after the pages have benn loaded
nameAndDesig = []
dept = []
phone = []

for ele in driver.find_elements_by_css_selector("div.small-12.medium-9.large-9.columns.listing-details"):
    # Scrape name and designation
    try:
        nameAndDesig.append(ele.find_element_by_css_selector("h2").text)
    except:
        nameAndDesig.append("na")
        
    # Scrap[e department
    try:
        dept.append(ele.find_element_by_css_selector("strong").text)
    except:
        dept.append("na")
    
    # Scrape pnone no
    try:
        phone.append(ele.find_elements_by_css_selector("p")[-2].text)
    except:
        phone.append("na")

In [4]:
# Cleaning and feature engineering
# Extract name and designation from nameAndDesignation
name = list(map(lambda x: x.split("-")[0].strip(), nameAndDesig))
designation = list(map(lambda x: x.split("-")[-1].strip(), nameAndDesig))

# Create a dataframe
df = pd.DataFrame({
    "name":name,
    "designation":designation,
    "phone":phone
})

# Now just preview the df
df.head(10)

,name,designation,phone
0,"Sumner, Christian",Lecturer/Senior Lecturer,+44 (0)115 941 8418
1,"Lee, Deborah",Lecturer/Senior Lecturer,+44 (0)115 848 5588
2,"Hutton, Rob",Lecturer,+44 (0)115 848 5612
3,"Alldred, Pam",Professor in Social Work,+44 (0)115 848 8219
4,"Foard, Nick",Lecturer/Senior Lecturer,+44 (0)115 848 8137
5,"Sahin, Hakan",Lecturer,+44 (0)115 848 8037
6,"Mcfadden, Joanne",Lecturer/Senior Lecturer,+44 (0)115 848 5816
7,"Mignot, Philip",Lecturer/Senior Lecturer,+44 (0)115 848 4192
8,"Rudkin, Phil",Principal Lecturer,+44 (0)115 848 5570
9,"Sutton, Andy",Lecturer/Senior Lecturer,+44 (0)115 848 2273


In [5]:
# Finally create a csv file
df.to_csv(f"nottinghamTrentUniversityStaffs_{today}.csv", index=None)